In [46]:
import pandas as pd
import matplotlib
import requests
from datetime import timedelta

GitHubUserURLBase = "https://api.github.com/users/"
GitHubPublicEvents = "/events/public"

class GitHubUser:
    def __init__(self, username):
        self.username = username
    def get_github_user(self):
        UserInfo = requests.get(GitHubUserURLBase + self.username)
        UserInfoUsable = UserInfo.json()
        return(UserInfoUsable)
    def verify_github_user(self):
        UserInfo = self.get_github_user()
        if 'login' in UserInfo:
            return("User_Exists")
        else:
            raise ValueError("Not a valid GitHub user")
    def get_github_user_activity(self):
        UserActivity = requests.get(GitHubUserURLBase + self.username + GitHubPublicEvents ) # + "?page=" + PageStr
        UserActivityPage = UserActivity.json()
        return(UserActivityPage)

##create a method to plot the daily activity of a user week over week day over day then display it
def UserActivityFrame(userevents):
    ids = []
    types = []
    times = []
    for activity in userevents:
        ids.append(activity['id'])
        types.append(activity['type'])
        times.append(activity['created_at'])
    df = pd.DataFrame({'id' : ids,
         'type' : types,
         'created_at' : times})
    df['created_at'] = pd.to_datetime(df['created_at'])
    df['created_at'] + timedelta(hours = 8)
    return(df)

# df['hour_of_day'] = df.created_at.dt.hour
# df['created_at'] + timedelta(hours = 8)
# print(df)

Detaer = GitHubUser("detaer")
DetaerEvents = Detaer.get_github_user_activity()
print(UserActivityFrame(DetaerEvents))

##create a method to plot the hourly activity of a user day over day then display it

##create main - this will need to take user input of the git hub username
##TODO take the input as a string (can't figure this out right now)
# def main():
#     UserName = input("Enter the GitHub username of the person you want to creep on:")
#     User = GitHubUser(UserName)
#     ##do a try except to throw an error if the user isn't real, if the user is real, grab their public activity
#     try:
#         TestUser = User.verify_github_user()
        
#     except:
#         print("User is not a valid github user!")    

# if __name__ == '__main__':
#     main()


            id       type          created_at
0   8998757717  PushEvent 2019-02-03 01:19:45
1   8998725844  PushEvent 2019-02-03 00:56:14
2   8998031924  PushEvent 2019-02-02 18:00:27
3   8990878713  PushEvent 2019-02-01 07:16:36
4   8847273697  PushEvent 2019-01-08 00:54:36
5   8826285915  PushEvent 2019-01-03 01:00:56
6   8826138946  PushEvent 2019-01-03 00:02:30
7   8782745390  PushEvent 2018-12-20 01:49:21
8   8782632252  PushEvent 2018-12-20 01:13:05
9   8776236396  PushEvent 2018-12-19 03:08:32
10  8776073669  PushEvent 2018-12-19 02:17:14
11  8769161457  PushEvent 2018-12-18 02:33:30
12  8769093296  PushEvent 2018-12-18 02:12:07
13  8768682256  PushEvent 2018-12-18 00:06:14
14  8768678927  PushEvent 2018-12-18 00:05:15
15  8768538023  PushEvent 2018-12-17 23:26:33
16  8768524198  PushEvent 2018-12-17 23:23:00
17  8745419501  PushEvent 2018-12-13 02:39:52
18  8745090090  PushEvent 2018-12-13 00:59:59
19  8738690981  PushEvent 2018-12-12 04:34:32
20  8730744913  PushEvent 2018-12-

In [41]:
ids = []
types = []
times = []
for activity in detaer.get_github_user_activity():
    ids.append(activity['id'])
    types.append(activity['type'])
    times.append(activity['created_at'])
    
df = pd.DataFrame({'id' : ids,
         'type' : types,
         'created_at' : times})

NameError: name 'detaer' is not defined

In [42]:
`df['created_at'] = pd.to_datetime(df['created_at'])`
`df['hour_of_day'] = df.created_at.dt.hour`

SyntaxError: invalid syntax (<ipython-input-42-0db7170d10ef>, line 1)

In [43]:
from datetime import timedelta
df['created_at'] + timedelta(hours = 8)

NameError: name 'df' is not defined

In [ ]:
%matplotlib inline